In [169]:
from snowflake.snowpark import Session
from snowflake.snowpark.functions import col, to_date, coalesce, lit, mode, when, count, max
import pandas as pd
from datetime import datetime, timedelta
from utils.session import get_cached_session, get_session
from datetime import date, timedelta

In [2]:
session = get_session()

In [131]:
ip_df.limit(10).to_pandas()

,IP_ITEM_ID,IP_ITEM_PRICE_ID
0,843866,238089071
1,2625158,238082714
2,3093487,238104300
3,1165091,238058355
4,1095668,238096285
5,1200488,238078283
6,1164102,238087651
7,1094466,238049142
8,1735268,238075537
9,1095627,238107522


In [173]:
ip_df = session.table('edl.phq.item_price').filter(
    (col('ip_start_date') <= default_date) &
    (coalesce(to_date(col('IP_END_DATE')), lit('9999-12-31')) >= default_date)
).select(
    col('item_id').alias('ip_item_id'),
    col('item_price_id').alias('item_price_id'),
    col('v_id').alias('v_id'),
    col('ip_unit_price').alias('ip_unit_price'),
    col('ip_price_multiple').alias('ip_price_multiple'),
    col('ip_start_date').alias('ip_start_date'),
    col('ip_end_date').alias('ip_end_date'),
    col('store_id').alias('ip_store_nbr'),
    col('pt_type').alias('ip_pt_type')
)

im_df = session.table('edl.phq.item_master').select(
    col('item_id').alias('im_item_id'),
    col('upc_ean').alias('upc_ean')
)

item_df = session.table('edw.rtl.retail_item_vw').select(
    col('product_upc').alias('product_upc'),
    col('mdse_grp_key').alias('mdse_grp_key'),
    col('mdse_catgy_key').alias('mdse_catgy_key'),
    col('item_description').alias('item_description')
)

zg_df = session.table('edl.phq.sn_rev_zonegrp').filter(
    (col('zonegroupcode') == 1) &
    (col('zonecode') == 1)
).select(
    col('zonecode').alias('zonecode'),
    col('zonename').alias('zonename'),
    col('storecode').alias('zg_store_nbr')
)

pt_df = session.table('edl.phq.price_type').select(
    col('pt_type').alias('pt_pt_type'),
    col('description').alias('pt_description')
)

df = ip_df.join(
    zg_df,
    zg_df['zg_store_nbr'] == ip_df['ip_store_nbr'],
    how = 'inner'
).join(
    pt_df,
    ip_df['ip_pt_type'] == pt_df['pt_pt_type'],
    how = 'inner'
)

df = df.group_by(
    col('ip_item_id'),
    col('v_id'),
    col('zonecode')
    ).agg(
    mode(when(col('ip_pt_type') == 1, col('ip_unit_price'))).alias('unit_price'),
    mode(when(col('ip_pt_type') == 1, col('ip_price_multiple'))).alias('price_multiple'),
    mode(when(col('ip_pt_type') == 1, col('ip_start_date'))).alias('start_date'),
    mode(when(col('ip_pt_type') == 1, col('ip_end_date'))).alias('end_date'),
    count(when(col('ip_pt_type') == 1, lit(1))).alias('store_count'),

    mode(when(col('ip_pt_type') != 1, col('pt_description'))).alias('promo_type'),
    mode(when(col('ip_pt_type') != 1, col('ip_unit_price'))).alias('promo_unit_price'),
    mode(when(col('ip_pt_type') != 1, col('ip_price_multiple'))).alias('promo_price_multiple'),
    mode(when(col('ip_pt_type') != 1, col('ip_start_date'))).alias('promo_start_date'),
    mode(when(col('ip_pt_type') != 1, col('ip_end_date'))).alias('promo_end_date'),
    count(when(col('ip_pt_type') != 1, lit(1))).alias('promo_store_count')
    )


In [174]:
df = df.to_pandas()


In [177]:
zg_df.to_pandas()

,ZONECODE,ZONENAME,ZG_STORE_NBR
0,1,SB AND ELKHART,2304
1,1,SB AND ELKHART,2306
2,1,SB AND ELKHART,2310
3,1,SB AND ELKHART,2311
4,1,SB AND ELKHART,2312
5,1,SB AND ELKHART,2314
6,1,SB AND ELKHART,2315
7,1,SB AND ELKHART,2316
8,1,SB AND ELKHART,2317
9,1,SB AND ELKHART,2324


In [117]:
len(df)

7374294

In [74]:
def prefix_columns(df, prefix):
    for col_name in df.columns:
        df = df.with_column_renamed(col_name, f"{prefix}_{col_name}")
    return df

ip_df = session.table('edl.phq.item_price')
irpd_df = session.table('edl.phq.item_retail_price_dt')
im_df = session.table('edl.phq.item_master')
pt_df = session.table('edl.phq.price_type')
zg_df = session.table('edl.phq.sn_rev_zonegrp')
item_df = session.table('edw.rtl.retail_item_vw')

ip_df = prefix_columns(ip_df, 'ip')
irpd_df = prefix_columns(irpd_df, 'irpd')
im_df = prefix_columns(im_df, 'im')
pt_df = prefix_columns(pt_df, 'pt')
zg_df = prefix_columns(zg_df, 'zg')
item_df = prefix_columns(item_df, 'item')

#joins
df = irpd_df.join(
    ip_df,
    irpd_df['irpd_item_price_id'] == ip_df['ip_item_price_id'],
    how = 'inner'
).join(
    im_df,
    ip_df['ip_item_id'] == im_df['im_item_id'],
    how = 'inner'
).join(
    item_df,
    im_df['im_upc_ean'] == item_df['item_product_upc'],
    how = 'inner'
).join(
    pt_df,
    irpd_df['irpd_pt_type'] == pt_df['pt_pt_type'],
    how = 'inner'
).join(
    zg_df,
    irpd_df['irpd_store_id'] == zg_df['zg_storecode']
)

#filters
c_df = df.filter(
    (irpd_df['irpd_ip_start_date'] <= default_date) &
    (irpd_df['irpd_ip_end_date'] >= default_date) &
    #(pt_df['pt_pt_type'] == '1') &
    (ip_df['ip_price_strategy'] != 0) &
    (zg_df['zg_zonecode'] == 1) &
    (zg_df['zg_zonegroupcode'] == 1)
).select(
    zg_df['zg_zonename'],
    item_df['item_product_upc'],
    irpd_df['irpd_v_id'],
    pt_df['pt_description'],
    irpd_df['irpd_ip_unit_price'],
    irpd_df['irpd_ip_price_multiple'],
    irpd_df['irpd_ip_start_date'],
    irpd_df['irpd_ip_end_date']
)
t_df = df.filter(
    (irpd_df['irpd_ip_start_date'] <= default_date) &
    (irpd_df['irpd_ip_end_date'] >= default_date) &
    #(pt_df['pt_pt_type'] == '1') &
    (ip_df['ip_price_strategy'] != 0) &
    (zg_df['zg_zonecode'] == 1) &
    (zg_df['zg_zonegroupcode'] == 1)
).select(
    zg_df['zg_zonename'],
    item_df['item_product_upc'],
    irpd_df['irpd_v_id'],
    pt_df['pt_description'],
    irpd_df['irpd_ip_unit_price'],
    irpd_df['irpd_ip_price_multiple'],
    irpd_df['irpd_ip_start_date'],
    irpd_df['irpd_ip_end_date']
)

#aggregations
result_df = df.group_by(
    zg_df['zg_zonename'],
    item_df['item_product_upc'],
    irpd_df['irpd_v_id']
).agg(
    mode(when(pt_df['pt_description'] == 'Regular', irpd_df['irpd_ip_unit_price'])).alias('IP_UNIT_PRICE'),
    mode(when(pt_df['pt_description'] == 'Regular', irpd_df['irpd_ip_price_multiple'])).alias('IP_PRICE_MULTIPLE'),
    mode(when(pt_df['pt_description'] == 'Regular', irpd_df['irpd_ip_start_date'])).alias('IP_START_DATE'),
    mode(when(pt_df['pt_description'] == 'Regular', irpd_df['irpd_ip_end_date'])).alias('IP_END_DATE'),
    count(when(pt_df['pt_description'] == 'Regular', lit(1))).alias('STORE_COUNT'),

    mode(when(pt_df['pt_description'] != 'Regular', pt_df['pt_description'])).alias('PROMO_TYPE'),
    mode(when(pt_df['pt_description'] != 'Regular', irpd_df['irpd_ip_unit_price'])).alias('PROMO_UNIT_PRICE'),
    mode(when(pt_df['pt_description'] != 'Regular', irpd_df['irpd_ip_price_multiple'])).alias('PROMO_PRICE_MULTIPLE'),
    mode(when(pt_df['pt_description'] != 'Regular', irpd_df['irpd_ip_start_date'])).alias('PROMO_START_DATE'),
    mode(when(pt_df['pt_description'] != 'Regular', irpd_df['irpd_ip_end_date'])).alias('PROMO_END_DATE'),
    count(when(pt_df['pt_description'] != 'Regular', lit(1))).alias('PROMO_STORE_COUNT')
)


In [ ]:
ip_df = session.table('edl.phq.item_price').select(
    col('item_id'),
    col('item_price_id'),
    col('v_id'),
    col('ip_unit_price'),
    col('price_strategy')
)
irpd_df = session.table('edl.phq.item_retail_price_dt')

ip_df = ip_df.join(
    irpd_df,
    on = ip_df['item_price_id'] == irpd_df['item_price_id'],
    how = 'inner'
).filter(
    (irpd_df['ip_start_date'] <= default_date) &
    (irpd_df['ip_end_date'] >= default_date) &
    (ip_df['price_strategy'] != 0)
).select(
    ip_df['item_id'],
    ip_df['item_price_id'],
    irpd_df['v_id'].alias('irpd_v_id'),
    ip_df['v_id'].alias('ip_v_id'),
    ip_df['ip_unit_price'],
    irpd_df['ip_unit_price']
).filter(irpd_df['v_id'] != ip_df['v_id']).to_pandas()


In [73]:
ip_df

,l_0254_ITEM_ID,l_0254_ITEM_PRICE_ID,IRPD_V_ID,IP_V_ID,l_0254_IP_UNIT_PRICE,r_0255_IP_UNIT_PRICE
0,2251652,251057622,651,51,2.69,2.69
1,2251652,251057622,123,51,2.69,2.69
2,2251652,251057622,175,51,2.69,2.69
3,2251652,251057623,123,51,2.69,2.69
4,2251652,251057623,175,51,2.69,2.69


In [ ]:
df = irpd_df.join(
    ip_df,
    irpd_df['item_price_id'] == ip_df['item_price_id'],
    how = 'inner'
).join(
    im_df,
    ip_df['item_id'] == im_df['item_id'],
    how = 'inner'
).join(
    item_df,
    im_df['upc_ean'] == item_df['product_upc_13_cd'],
    how = 'inner'
)

df.show(100)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [35]:
df

In [11]:
today = date.today()
days_since_saturday = (today.weekday() - 5) % 7
default_date = today - timedelta(days=days_since_saturday)
default_date

datetime.date(2025, 8, 23)

In [2]:
session = get_cached_session()

2025-08-27 17:22:35.542 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:22:36.048 
  command:

    streamlit run c:\Users\212739\OneDrive - SpartanNash\Documents\Data\Streamlit\Pricing App\.venv\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-08-27 17:22:36.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:22:36.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:22:36.048 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:22:36.557 Thread 'Thread-7': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-27 17:22:36.563 Thread 'Thread-7': missing ScriptRunContext! This warning can be ignored when running in bare 

In [20]:
df = session.table("SBX_BIZ.MARKETING.T_ITEM").select(
        col('"Anchor Group ID"'),
        col('"Item Description"'),
        col('"Product UPC"'),
        col('"Unit Size"'),
        col('"Group ID"'),
        col('"Category ID"'),
        col('"Brand"')
    ).to_pandas()

In [15]:
item_prices_df = session.table('EDL.PHQ.ITEM_PRICE').filter(
    (default_date >= to_date(col('IP_START_DATE'))) & 
        (default_date <= coalesce(to_date(col('IP_END_DATE')), lit('9999-12-31'))) &
    (col('PRICE_STRATEGY') == 11) & 
    (col('RECORD_STATUS') != 3)
)

item_prices_df.show(10)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
item_prices_df = session.table('EDL.PHQ.ITEM_PRICE').filter(
    (col('PRICE_STRATEGY') == 11) & 
    (col('RECORD_STATUS') != 3)
)

item_prices_df.show(10)

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------